## Silver Layer: Cleaning & Normalization
- Deduplication
- Text normalization
- Bridge table construction

#### Design Dimension, Facts, and Composite Tables for Silver layer

ie. Game Dimension, Developer Dimension, Publisher Dimension, Category dimension, Description Dimension, Fact table with foreign keys

#### Create Silver layer schema objects

In [ ]:
USE SCHEMA SILVER;

-- Developer dimension
CREATE OR REPLACE TABLE DIM_DEVELOPER (
    DEVELOPER_KEY INT AUTOINCREMENT,
    DEVELOPER VARCHAR(500),
    PRIMARY KEY (DEVELOPER_KEY)
);

-- Publisher dimension
CREATE OR REPLACE TABLE DIM_PUBLISHER (
    PUBLISHER_KEY INT AUTOINCREMENT,
    PUBLISHER VARCHAR(500),
    PRIMARY KEY (PUBLISHER_KEY)
);

-- Category dimension
CREATE OR REPLACE TABLE DIM_CATEGORY (
    CATEGORY_KEY INT AUTOINCREMENT,
    CATEGORY VARCHAR(200),
    PRIMARY KEY (CATEGORY_KEY)
);

-- Genre dimension
CREATE OR REPLACE TABLE DIM_GENRE (
    GENRE_KEY INT AUTOINCREMENT,
    GENRE VARCHAR(200),
    PRIMARY KEY (GENRE_KEY)
);

-- Description dimension
CREATE OR REPLACE TABLE DIM_DESCRIPTION (
    APP_ID NUMBER,
    DETAILED_DESCRIPTION VARCHAR(16777216),
    ABOUT_THE_GAME VARCHAR(16777216),
    SHORT_DESCRIPTION VARCHAR(16777216),
    PRIMARY KEY (APP_ID)
);

-- Game dimension
CREATE OR REPLACE TABLE DIM_GAME (
    APP_ID NUMBER,
    NAME VARCHAR(500),
    RELEASE_DATE DATE,
    REQUIRED_AGE NUMBER,
    PLATFORMS VARCHAR(100),
    ENGLISH NUMBER,
    PRIMARY KEY (APP_ID)
);

-- Fact table (CORRECTED with all metrics columns)
CREATE OR REPLACE TABLE FACT_GAME_METRICS (
    FACT_KEY INT AUTOINCREMENT,
    APP_ID NUMBER,
    DEVELOPER_KEY INT,
    PUBLISHER_KEY INT,
    PRICE NUMBER(10,2),
    POSITIVE_RATINGS NUMBER,
    NEGATIVE_RATINGS NUMBER,
    AVERAGE_PLAYTIME NUMBER,
    MEDIAN_PLAYTIME NUMBER,
    OWNERS VARCHAR(100),
    ACHIEVEMENTS NUMBER,
    PRIMARY KEY (FACT_KEY),
    FOREIGN KEY (APP_ID) REFERENCES DIM_GAME(APP_ID),
    FOREIGN KEY (DEVELOPER_KEY) REFERENCES DIM_DEVELOPER(DEVELOPER_KEY),
    FOREIGN KEY (PUBLISHER_KEY) REFERENCES DIM_PUBLISHER(PUBLISHER_KEY)
);

-- Bridge table for Game-Category (many-to-many)
CREATE OR REPLACE TABLE BRIDGE_GAME_CATEGORY (
    APP_ID NUMBER,
    CATEGORY_KEY INT,
    FOREIGN KEY (APP_ID) REFERENCES DIM_GAME(APP_ID),
    FOREIGN KEY (CATEGORY_KEY) REFERENCES DIM_CATEGORY(CATEGORY_KEY)
);

-- Bridge table for Game-Genre (many-to-many)
CREATE OR REPLACE TABLE BRIDGE_GAME_GENRE (
    APP_ID NUMBER,
    GENRE_KEY INT,
    FOREIGN KEY (APP_ID) REFERENCES DIM_GAME(APP_ID),
    FOREIGN KEY (GENRE_KEY) REFERENCES DIM_GENRE(GENRE_KEY)
);

#### Populate silver layer data from bronze layer maintaining data consistency

In [ ]:
USE SCHEMA SILVER;

-- Clean + Load Developer Dimension
TRUNCATE TABLE SILVER.DIM_DEVELOPER;
INSERT INTO SILVER.DIM_DEVELOPER (DEVELOPER)
SELECT DISTINCT
    INITCAP(TRIM("developer")) AS DEVELOPER
FROM BRONZE.STEAM_GAMES
WHERE "developer" IS NOT NULL
  AND TRIM("developer") <> ''
  AND UPPER(TRIM("developer")) NOT IN ('N/A', 'UNKNOWN', 'NONE', '');

-- Clean + Load Publisher Dimension
TRUNCATE TABLE SILVER.DIM_PUBLISHER;
INSERT INTO SILVER.DIM_PUBLISHER (PUBLISHER)
SELECT DISTINCT
    INITCAP(TRIM("publisher")) AS PUBLISHER
FROM BRONZE.STEAM_GAMES
WHERE "publisher" IS NOT NULL
  AND TRIM("publisher") <> ''
  AND UPPER(TRIM("publisher")) NOT IN ('N/A', 'UNKNOWN', 'NONE', '');

-- Clean + Load Category Dimension
TRUNCATE TABLE SILVER.DIM_CATEGORY;
INSERT INTO SILVER.DIM_CATEGORY (CATEGORY)
SELECT DISTINCT
    INITCAP(TRIM(cat.value)) AS CATEGORY
FROM BRONZE.STEAM_GAMES,
     LATERAL SPLIT_TO_TABLE("categories", ';') AS cat
WHERE cat.value IS NOT NULL
  AND TRIM(cat.value) <> ''
ORDER BY CATEGORY;

-- Clean + Load Genre Dimension
TRUNCATE TABLE SILVER.DIM_GENRE;
INSERT INTO SILVER.DIM_GENRE (GENRE)
SELECT DISTINCT
    INITCAP(TRIM(gen.value)) AS GENRE
FROM BRONZE.STEAM_GAMES,
     LATERAL SPLIT_TO_TABLE("genres", ';') AS gen
WHERE gen.value IS NOT NULL
  AND TRIM(gen.value) <> ''
ORDER BY GENRE;

-- Clean + Load Description Dimension
TRUNCATE TABLE SILVER.DIM_DESCRIPTION;
INSERT INTO SILVER.DIM_DESCRIPTION (APP_ID, DETAILED_DESCRIPTION, ABOUT_THE_GAME, SHORT_DESCRIPTION)
SELECT DISTINCT
    "steam_appid" AS APP_ID,
    TRIM("detailed_description") AS DETAILED_DESCRIPTION,
    TRIM("about_the_game") AS ABOUT_THE_GAME,
    TRIM("short_description") AS SHORT_DESCRIPTION
FROM BRONZE.STEAM_DESCRIPTIONS
WHERE "steam_appid" IS NOT NULL;

-- Clean + Load Game Dimension
TRUNCATE TABLE SILVER.DIM_GAME;
INSERT INTO SILVER.DIM_GAME (APP_ID, NAME, RELEASE_DATE, REQUIRED_AGE, PLATFORMS, ENGLISH)
SELECT DISTINCT
    "appid" AS APP_ID,
    INITCAP(TRIM("name")) AS NAME,
    "release_date" AS RELEASE_DATE,
    NULLIF("required_age", 0) AS REQUIRED_AGE,
    LOWER(TRIM("platforms")) AS PLATFORMS,
    CASE WHEN "english" = 1 THEN 1 ELSE 0 END AS ENGLISH
FROM BRONZE.STEAM_GAMES
WHERE "appid" IS NOT NULL
  AND TRIM("name") <> '';

-- Clean + Build Fact Table
TRUNCATE TABLE SILVER.FACT_GAME_METRICS;
INSERT INTO SILVER.FACT_GAME_METRICS (
    APP_ID, 
    DEVELOPER_KEY, 
    PUBLISHER_KEY,
    PRICE, 
    POSITIVE_RATINGS, 
    NEGATIVE_RATINGS,
    AVERAGE_PLAYTIME, 
    MEDIAN_PLAYTIME, 
    OWNERS,
    ACHIEVEMENTS
)
SELECT 
    g."appid" AS APP_ID,
    dev.DEVELOPER_KEY,
    pub.PUBLISHER_KEY,
    CASE WHEN g."price" >= 0 THEN g."price" ELSE NULL END AS PRICE,
    CASE WHEN g."positive_ratings" >= 0 THEN g."positive_ratings" ELSE NULL END AS POSITIVE_RATINGS,
    CASE WHEN g."negative_ratings" >= 0 THEN g."negative_ratings" ELSE NULL END AS NEGATIVE_RATINGS,
    CASE WHEN g."average_playtime" >= 0 THEN g."average_playtime" ELSE NULL END AS AVERAGE_PLAYTIME,
    CASE WHEN g."median_playtime" >= 0 THEN g."median_playtime" ELSE NULL END AS MEDIAN_PLAYTIME,
    TRIM(g."owners") AS OWNERS,
    CASE WHEN g."achievements" >= 0 THEN g."achievements" ELSE NULL END AS ACHIEVEMENTS
FROM BRONZE.STEAM_GAMES g
LEFT JOIN SILVER.DIM_DEVELOPER dev 
    ON INITCAP(TRIM(g."developer")) = dev.DEVELOPER
LEFT JOIN SILVER.DIM_PUBLISHER pub 
    ON INITCAP(TRIM(g."publisher")) = pub.PUBLISHER
WHERE g."appid" IS NOT NULL;

-- Populate Bridge Table: Game-Category
TRUNCATE TABLE SILVER.BRIDGE_GAME_CATEGORY;
INSERT INTO SILVER.BRIDGE_GAME_CATEGORY (APP_ID, CATEGORY_KEY)
SELECT DISTINCT
    g."appid" AS APP_ID,
    cat.CATEGORY_KEY
FROM BRONZE.STEAM_GAMES g,
     LATERAL SPLIT_TO_TABLE(g."categories", ';') AS cat_split
INNER JOIN SILVER.DIM_CATEGORY cat 
    ON INITCAP(TRIM(cat_split.value)) = cat.CATEGORY
WHERE g."appid" IS NOT NULL
  AND cat_split.value IS NOT NULL
  AND TRIM(cat_split.value) <> '';

-- Populate Bridge Table: Game-Genre
TRUNCATE TABLE SILVER.BRIDGE_GAME_GENRE;
INSERT INTO SILVER.BRIDGE_GAME_GENRE (APP_ID, GENRE_KEY)
SELECT DISTINCT
    g."appid" AS APP_ID,
    gen.GENRE_KEY
FROM BRONZE.STEAM_GAMES g,
     LATERAL SPLIT_TO_TABLE(g."genres", ';') AS gen_split
INNER JOIN SILVER.DIM_GENRE gen 
    ON INITCAP(TRIM(gen_split.value)) = gen.GENRE
WHERE g."appid" IS NOT NULL
  AND gen_split.value IS NOT NULL
  AND TRIM(gen_split.value) <> '';

-- Preview with joins
SELECT 
    f.APP_ID,
    g.NAME,
    dev.DEVELOPER,
    pub.PUBLISHER,
    f.PRICE,
    f.POSITIVE_RATINGS,
    f.OWNERS
FROM SILVER.FACT_GAME_METRICS f
JOIN SILVER.DIM_GAME g ON f.APP_ID = g.APP_ID
LEFT JOIN SILVER.DIM_DEVELOPER dev ON f.DEVELOPER_KEY = dev.DEVELOPER_KEY
LEFT JOIN SILVER.DIM_PUBLISHER pub ON f.PUBLISHER_KEY = pub.PUBLISHER_KEY
LIMIT 20;